In [2]:
require(data.table)
require(tidyverse)

## 1. Read data 

In [3]:
res <- readRDS('../dat/output.peaks.by.cells.count.sparse.Rdata')

In [4]:
str(res)

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand



Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:38271863] 22 286 465 666 952 1191 1288 1804 2221 2414 ...
  ..@ p       : int [1:14966] 0 1160 2722 5504 6194 13034 16851 19353 23914 26970 ...
  ..@ Dim     : int [1:2] 255590 14965
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:255590] "chr1_100009896_100010395" "chr1_100014623_100015122" "chr1_100017609_100018108" "chr1_100023384_100023883" ...
  .. ..$ : chr [1:14965] "Islet1-fresh_AGACACCTAAGAGGCAAAGGAGTACCTAT" "Islet1-fresh_AGACACCTAAGAGGCAAAGGAGTAGGCTC" "Islet1-fresh_AGACACCTAAGAGGCAAAGGAGTAGGTTG" "Islet1-fresh_AGACACCTAAGAGGCAAAGGCTATATAGA" ...
  ..@ x       : num [1:38271863] 2 1 2 1 2 2 2 1 2 1 ...
  ..@ factors : list()


In [3]:
dat.mat <- fread('../dat/1908/all.distal.long.matrix.csv')
dim(dat.mat)
head(dat.mat)
#dat.mat$cell <- NULL #no need cell id 

[1] 45314787        3

distal_peak,barcode,cluster
1_10216_10512,Islet3-fresh_CTGAAGCTTGCAGCTACTCTCTATTATAGCCT,beta_2
1_237657_237858,Islet3-fresh_CTGAAGCTTGCAGCTACTCTCTATTATAGCCT,beta_2
1_967851_968930,Islet3-fresh_CTGAAGCTTGCAGCTACTCTCTATTATAGCCT,beta_2
1_967851_968930,Islet3-fresh_CTGAAGCTTGCAGCTACTCTCTATTATAGCCT,beta_2
1_1004053_1005548,Islet3-fresh_CTGAAGCTTGCAGCTACTCTCTATTATAGCCT,beta_2
1_1004053_1005548,Islet3-fresh_CTGAAGCTTGCAGCTACTCTCTATTATAGCCT,beta_2


 Concepts: 
1. `promoter region`: -500bp + 500bp for all TSS in gencode 
2. `promoter Peaks`: peaks that overlap promoter region

In [4]:
dat.mat <- dat.mat %>% distinct()
dim(dat.mat)

[1] 22097911        3

In [5]:
dat.mat[,cluster:=sub('(_1)|(_2)','',cluster)]

In [6]:
# all cells 
dat.all.cells <- table((dat.mat[,-"distal_peak"]%>%distinct())$clust)
dat.all.cells


      alpha        beta       delta endothelial    exocrine       gamma 
       5594        7170         718         157         131         206 
      glial      immune    stellate 
         39          71         153 

In [6]:
sum(dat.all.cells)

ERROR: Error in eval(expr, envir, enclos): object 'dat.all.cells' not found


In [ ]:
# how many peaks
length(unique(dat.mat$distal_peak))

## 2 Prepare data


Peak|n_cells|celltype 
--- | --- | ---


In [ ]:
# prepare data
dat.mat<- dat.mat[,.N,by=.(distal_peak,cluster)]%>%arrange(distal_peak,cluster)

In [ ]:
head(dat.mat)

## 3 Fisher's test one vs other 

In [16]:
tr = "1_100009936_100010354"
celltypes <- "alpha"
#celltypes <- 'beta'
test.dat <- dat.mat %>% filter(distal_peak == tr)
test.dat <- test.dat %>% mutate(cluster = ifelse(cluster == celltypes, celltypes, 
    "other")) %>% group_by(distal_peak, cluster) %>% summarise(N = sum(N))
test.dat
table.res <- test.dat$N
names(table.res) <- test.dat$cluster
# handle if 0 for one subtype
a = setdiff(celltypes, names(table.res))
table.res[a] <- 0
table.res
all_test_cells = as.numeric(dat.all.cells[celltypes])
other_cells = as.numeric(sum(dat.all.cells) - dat.all.cells[celltypes])
test.tab <- matrix(c(table.res[celltypes], table.res["other"], all_test_cells - table.res[celltypes], 
    other_cells - table.res["other"]), byrow = T, nrow = 2, dimnames = list(expressed = c("Yes", 
    "No"), subtype = c(celltypes, "other")))
test.tab
test.tab[1]
test.tab[2]

(f1 <- test.tab[1]/(test.tab[1] + test.tab[2]))
(f2 <- test.tab[3]/(test.tab[3] + test.tab[4]))
(l <- ifelse(f1 > f2, "greater", "less"))
test.res <- fisher.test(test.tab, alternative = l)
test.res
res <- list(pval = test.res$p.value, odds = test.res$estimate, type_frac = f1, other_frac = f2)
t(res)

(test.res <- fisher.test(test.tab))

distal_peak,cluster,N
1_100009936_100010354,alpha,23
1_100009936_100010354,other,235


alpha other 
   23   235

,alpha,other
Yes,23,235
No,5571,8410


[1] 23

[1] 5571

...

[1] 0.02718334

[1] "less"


	Fisher's Exact Test for Count Data

data:  test.tab
p-value < 2.2e-16
alternative hypothesis: true odds ratio is less than 1
95 percent confidence interval:
 0.0000000 0.2138827
sample estimates:
odds ratio 
 0.1477894 


pval,odds,type_frac,other_frac
1.066244e-28,0.1477894,0.004111548,0.02718334



	Fisher's Exact Test for Count Data

data:  test.tab
p-value < 2.2e-16
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 0.09173139 0.22746492
sample estimates:
odds ratio 
 0.1477894 


function

In [17]:
x = "alpha"
dat.input = dat.mat %>% mutate(cluster = ifelse(cluster == celltypes, celltypes, 
    "other")) %>% group_by(distal_peak, cluster) %>% summarise(N = sum(N))

fun.testPerTr_OneVsOther <- function(tr = "1_100009936_100010354", celltypes = x, 
    dat = dat.input) {
    
    test.dat <- dat %>% filter(distal_peak == tr)
    table.res <- test.dat$N
    names(table.res) <- test.dat$cluster
    
    # handle if 0 for one subtype
    a = setdiff(celltypes, names(table.res))
    table.res[a] <- 0
    
    all_test_cells = as.numeric(dat.all.cells[celltypes])
    other_cells = as.numeric(sum(dat.all.cells) - dat.all.cells[celltypes])
    test.tab <- matrix(c(table.res[celltypes], table.res["other"], all_test_cells - 
        table.res[celltypes], other_cells - table.res["other"]), byrow = T, nrow = 2, 
        dimnames = list(expressed = c("Yes", "No"), subtype = c(celltypes, "other")))
    f1 <- test.tab[1]/(test.tab[1] + test.tab[2])
    f2 <- test.tab[3]/(test.tab[3] + test.tab[4])
    l <- ifelse(f1 > f2, "greater", "less")
    test.res <- fisher.test(test.tab, alternative = l)
    res <- list(distal_peak = tr, pval = test.res$p.value, odds = test.res$estimate, 
        type1_frac = f1, type2_frac = f2)
    res
}

## 


system.time({
    res = fun.testPerTr_OneVsOther()
})
t(res)

   user  system elapsed 
  7.316   0.035   7.353 

distal_peak,pval,odds,type1_frac,type2_frac
1_100009936_100010354,1.066244e-28,0.1477894,0.004111548,0.02718334


In [49]:
x = "alpha"
fun.testPerTr_OneVsOther <- function(tr = "1_100009936_100010354", celltypes = x, 
    dat = dat.mat) {
    
    test.dat <- dat %>% filter(distal_peak == tr) %>% mutate(cluster = ifelse(cluster == 
        celltypes, celltypes, "other")) %>% group_by(distal_peak, cluster) %>% summarise(N = sum(N))
    table.res <- test.dat$N
    names(table.res) <- test.dat$cluster
    
    # handle if 0 for one subtype
    a = setdiff(c(x, "other"), names(table.res))
    table.res[a] <- 0
    
    all_test_cells = as.numeric(dat.all.cells[celltypes])
    other_cells = as.numeric(sum(dat.all.cells) - dat.all.cells[celltypes])
    test.tab <- matrix(c(table.res[celltypes], table.res["other"], all_test_cells - 
        table.res[celltypes], other_cells - table.res["other"]), byrow = T, nrow = 2, 
        dimnames = list(expressed = c("Yes", "No"), subtype = c(celltypes, "other")))
    f1 <- test.tab[1]/(test.tab[1] + test.tab[2])
    f2 <- test.tab[3]/(test.tab[3] + test.tab[4])
    l <- ifelse(f1 > f2, "greater", "less")
    test.res <- fisher.test(test.tab, alternative = l)
    res <- list(distal_peak = tr, pval = test.res$p.value, odds = test.res$estimate, 
        type_frac = f1, other_frac = f2, celltype = celltypes)
    res
}


system.time({
    res = fun.testPerTr_OneVsOther()
})
t(res)

   user  system elapsed 
  0.045   0.001   0.046 

distal_peak,pval,odds,type_frac,other_frac,celltype
1_100009936_100010354,1.066244e-28,0.1477894,0.004111548,0.02718334,alpha


In [20]:
# estimate time
cat("n peaks:")
(npeaks = sapply(names(dat.all.cells), function(x) length(dat.mat %>% filter(cluster == 
    x) %>% pull(distal_peak) %>% unique())))
cat("estimated time using 10 cores:", 0.05 * length(unique(dat.mat$distal_peak)) * 
    4/10/3600, "hr")

n peaks:

alpha        beta       delta endothelial    exocrine       gamma 
     207977      210493      157512       60043       69332       92269 
      glial      immune    stellate 
      33378       33125       83508

estimated time using 10 cores: 1.222378 hr

In [14]:

# estimate time
cat("n peaks:")
(npeaks = sapply( names(dat.all.cells),function(x) 
    length(dat.mat%>% filter(cluster == x)%>%pull(distal_peak)%>%unique())))
cat("estimated time using 10 cores:", 0.05*sum(npeaks)/10/3600 ,'hr')                 

n peaks:

alpha        beta       delta endothelial    exocrine       gamma 
     207977      210493      157512       60043       69332       92269 
      glial      immune    stellate 
      33378       33125       83508

estimated time using 10 cores: 1.316163 hr

In [21]:
all.tr <- dat.mat %>%  pull(distal_peak) %>% unique()

In [23]:
require(parallel)

res <- list()

all.tr <- dat.mat %>% pull(distal_peak) %>% unique()
# time consumming task system.time(for (x in names(dat.all.cells)) {
system.time(for (x in c("alpha", "beta", "delta", "gamma")) {
    print(x)
    res[[x]] <- do.call(rbind, mclapply(all.tr, function(trr) t(fun.testPerTr_OneVsOther(tr = trr, 
        celltypes = x)), mc.cores = 10))
})
saveRDS(object = do.call(rbind, res), file = "../dat/1908/res.distal.peaks.fisher.oneVother.celltype_2.Rds")

Loading required package: parallel


[1] "alpha"
[1] "beta"
[1] "delta"
[1] "gamma"


     user    system   elapsed 
28164.492  2054.553  3077.476 